In [6]:
import quandl
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing, cross_validation
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
quandl.ApiConfig.api_key = 'DEN8cmx6MYrtzyeUdDeh'

In [8]:
%%time
tickers = ['FB','GOOGL','AAPl','AMZN','TSLA','MSFT','AMD','INTC','T','NFLX']
Validate = []
Train = []
for ticker in tickers:
    df = quandl.get("WIKI/"+ticker)
    df = df.iloc[-500:]
    func = ['mean','std','skew','kurt']
    time = [2,4,7,30,60,120,250]
    Z = pd.DataFrame()
    for f in func:
        for t in time:
            aux = pd.DataFrame()
            for i in range(249):
                aux[df.index[i+251]] = list(df.iloc[i-t+251:i+251]['Adj. Close'])
            Z[f+str(t)] = aux.apply(f,axis=0)  
    Z.drop(['skew2','kurt2'],axis=1, inplace =True)
    W = pd.merge(Z, df[['Adj. Close']],left_index=True,right_index=True)
    X = W.drop(['Adj. Close'],1).fillna(0)
    y = W[['Adj. Close']]
    mm1 = MinMaxScaler()
    mm2 = MinMaxScaler()
    mm1.fit(X)
    mm2.fit(y)
    Xm = pd.DataFrame(mm1.transform(X))
    ym = pd.DataFrame(mm2.transform(y))
    Xt, Xv, yt, yv = cross_validation.train_test_split(Xm, ym, test_size = 0.3)
    model =  GradientBoostingRegressor()
    model.fit(Xm,ym)
    print ticker
    print 'ACC Validate {:.2%}'.format(model.score(Xv,yv))
    print 'ACC Train    {:.2%}'.format(model.score(Xt,yt))
    Validate.append(model.score(Xv,yv))
    Train.append(model.score(Xt,yt))
print 'Promedio Validate  {:.2%}'.format(np.array(Validate).mean())
print 'Promedio Train     {:.2%}'.format(np.array(Train).mean())

FB
ACC Validate 99.67%
ACC Train    99.58%
GOOGL
ACC Validate 99.79%
ACC Train    99.73%
AAPl
ACC Validate 99.43%
ACC Train    99.69%
AMZN
ACC Validate 99.95%
ACC Train    99.94%
TSLA
ACC Validate 98.11%
ACC Train    98.35%
MSFT
ACC Validate 99.90%
ACC Train    99.92%
AMD
ACC Validate 98.62%
ACC Train    98.51%
INTC
ACC Validate 99.90%
ACC Train    99.90%
T
ACC Validate 99.00%
ACC Train    98.87%
NFLX
ACC Validate 99.93%
ACC Train    99.94%
Promedio Validate  99.43%
Promedio Train     99.44%
CPU times: user 1min 35s, sys: 759 ms, total: 1min 35s
Wall time: 1min 58s
